# Fine-tuning an Image Model

This notebook is part of [Lab 2](https://github.com/ee292d/labs/blob/main/lab2/README.md) for the [EE292D Edge ML class](https://ee292d.github.io/) at Stanford. It shows how to prepare an image classification model for your Pi, and fine-tune it to recognize different kinds of objects. It should take around 30 minutes to complete.

If you're not already in Google's Colab environment (or another notebook client) [click here to open it in Colab](https://colab.research.google.com/github/ee292d/labs/blob/main/lab2/notebook.ipynb).

## Install Ultralytics

I'm a big fan of the [Ultralytics project](https://github.com/ultralytics/ultralytics) that provides a free, open source wrapper for some very efficient image models from the YOLO family. We'll be using their Python module to download base models, fine tune a model on custom data, and export the result to TensorFlow Lite to run on the Raspberry Pi.

When you run this snippet you should see a lot of log messages about installation, followed by a "Setup complete" message. If the GPU selection in the previous step worked, you'll see something like `CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)` on the first line, indicating you have an instance with a GPU. It should take a couple of minutes to complete.

In [4]:
%pip install ultralytics

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.38 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 29.3/201.2 GB disk)


## Download and Run an Image Classifier Model

Ultralytics is based around the YOLO family of image models. These are designed to be efficient, come in a range of sizes, and offer [different kinds of prediction](https://docs.ultralytics.com/tasks/). To get started we'll be using version 8 of the classification model (indicated by the `cls` suffix) in the nano size (the `n` suffix), which is trained on the ImageNet dataset.

In these two lines we'll download and create the model, and then run it on a test image. This is a picture of Admiral Grace Hopper, and since there are no person labels in ImageNet we expect to see `military_uniform` as the top label.

In [5]:
from ultralytics import YOLO

model = YOLO("yolov8n-cls.pt")
_ = model("https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg")

100%|██████████| 5.30M/5.30M [00:00<00:00, 158MB/s]


100%|██████████| 59.9k/59.9k [00:00<00:00, 1.95MB/s]


image 1/1 /content/grace_hopper.jpg: 224x224 military_uniform 0.86, bolo_tie 0.05, bow_tie 0.03, suit 0.01, jersey 0.01, 3.5ms
Speed: 67.4ms preprocess, 3.5ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


## Export your Model

Ultralytics does work on a Raspberry Pi, but you'll usually get lower latency and compatibility with more edge devices if you use a dedicated inference framework like TensorFlow Lite or OpenVINO. Thankfully Ultralytics can export its models to many different formats. The cell below will write out the TensorFlow Lite 8-bit quantized version of the model to `save_dir` + `"weights/best_saved_model/best_int8.tflite"`. Open up the file browser from the panel in the left sidebar, navigate to the folder, and download the file to your laptop to continue with the rest of the lab.

*Note: You'll currently see an error* `cannot initialize type "StatusCode"` *at the end when you run this cell, but if you look for the file it should still be there despite [this problem](https://github.com/ultralytics/ultralytics/issues/5161).*



In [8]:
flower_model.export(format='tflite', int8=True)

Ultralytics YOLOv8.1.38 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'runs/classify/train/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 5) (2.8 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnx2tf>=1.15.4,<=1.17.5', 'sng4onnx>=1.0.1', 'onnxsim>=0.4.33', 'onnx_graphsurgeon>=0.3.26', 'tflite_support', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.4/400.4 kB 315.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 229.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 216.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 MB 253.4 MB/s et

100%|██████████| 1.11M/1.11M [00:00<00:00, 57.8MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 47.39file/s]


ONNX: starting export with onnx 1.16.0 opset 17...


ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.4s, saved as 'runs/classify/train/weights/best.onnx' (5.5 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.17.5...

Automatic generation of each OP name started ========================================
Automatic generation of each OP name complete!

Model loaded ========================================================================

Model conversion started ============================================================
saved_model output started ==========================================================
saved_model output complete!
Float32 tflite output complete!
Float16 tflite output complete!
Input signature information for quantization
signature_name: serving_default
input_name.0: images shape: (1, 224, 224, 3) dtype: <dtype: 'float32'>
Dynamic Range Quantization tflite output complete!
INT8 Quantization tflite output complete!
Full INT8 Quantization tflite output complete!
INT8 Quantization with int16

ImportError: generic_type: cannot initialize type "StatusCode": an object with that name is already defined